In [1]:
#Human pos classification with movenet and tensorflow lite
#preparation
#preparing all needed libraries
!pip install -q opencv-python

In [3]:
import csv
import cv2
import itertools
import numpy as np
import pandas as pd
import os
import sys
import tempfile
import tqdm


from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [5]:
#@title Functions to run pose estimation with MoveNet